In [1]:
# Imports
import pandas as pd
import requests
from bs4 import BeautifulSoup
from geopy.geocoders import Bing
geolocator = Bing(api_key='Insert API Key Here',user_agent='man_utd_travel')

In [2]:
# Checks to see if Wikipedia allows for this article to be downloaded.
wiki_page = 'https://en.wikipedia.org/wiki/2020%E2%80%9321_Manchester_United_F.C._season'
table_class="wikitable"
response=requests.get(wiki_page)
print(response.status_code)

200


In [3]:
# Parses through the wikipedia page for ALL tables by checking for the 'table' tag and 'wikitable' class name and stores them.
soup = BeautifulSoup(response.text, 'html.parser')
wiki_tables=soup.find_all('table',{'class':"wikitable"})

In [4]:
tables = []
for i in range(len(wiki_tables)):
    # Creates dataframes of the wikitables
    df = pd.read_html(str(wiki_tables[i]))
    df = pd.DataFrame(df[0])
    
    # The table column names are set as row 0. This moves that row to be column names.
    # NOTE: After having issues and reinstalling pandas/numpy, this was no longer needed.
    #new_header = df.iloc[0]
    #df = df[1:]
    #df.columns = new_header
    
    # Stores all the dataframes.
    tables.append(df)

# Removes league table, group stage table, and squad statistics tables from the list.
del tables[-5:]
del tables[2]

In [5]:
# Cleans up tables by removing unnecessary columns
for i in range(len(tables)):
    tables[i] = tables[i][['Date','Opponents', 'H / A']]
    tables[i].columns = tables[i].columns.str.replace('Opponents','Team')

In [6]:
for i in range(len(tables)):
    
    # Creates an empty list every iteration to hold the coordinates of every away stadium.
    latitude = []
    longitude = []
    
    for j in range(len(tables[i])):
        # Check to see if Man Utd are the home team. If so, skip as we know their home stadium.
        if tables[i]['H / A'][j] == 'H':
            print("Match:",j,"Man Utd are the home team\n")
            latitude.append(" ")
            longitude.append(" ")
            continue
        
        else:
            # Echoes the search request of the stadium location.
            print("Match:",j,"Searching:",tables[i]['Team'][j],"Football Club Stadium")
            location = geolocator.geocode((tables[i]['Team'][j],"Football Club Stadium"))
            
            # Echoes the coordinates found and adds them to the coordinates list.
            print("Location:",location,(location.latitude,location.longitude),"\n")
            latitude.append(location.latitude)
            longitude.append(location.longitude)
    
    # Prints the coordinate list that will be used
    print("Inserting coordinates into table")
    
    # Creates a new column for each table detailing the coordinates of the away team stadium.
    tables[i]['Latitude'] = latitude
    tables[i]['Longitude'] = longitude

# Variables holding the coordinates of Manchester United's Home stadium.
Old_Trafford = geolocator.geocode("Old Trafford Football Stadium")
Man_Utd_Latitude = Old_Trafford.latitude
Man_Utd_Longitude = Old_Trafford.longitude
    
# Remove all the home games, as the team will not travel, & resets the index of each row.
# Renames the 'H / A' column into 'Origin / Destination' and 'A' into 'Destination'.
# This will help to better understand the tables within Tableau.
for i in range(len(tables)):
    tables[i].drop(tables[i][tables[i]['H / A'] == 'H'].index, inplace = True)
    tables[i].reset_index(drop=True,inplace=True)
    tables[i].loc[tables[i]["H / A"] == 'A', 'H / A'] = 'Destination'
    tables[i].columns = tables[i].columns.str.replace('H / A','Origin / Destination')

# Inserts Manchester United's information into the tables, this will act as the 'Origin' of the travel between matchdays.
# The added 'Path ID' column will connect between the Origin and Destination points of Manchester United's travel.
for i in range(len(tables)):
    for j in range(len(tables[i])):
        try:
            utd = pd.DataFrame({'Date':tables[i]['Date'][j], 'Team':'Manchester United' ,'Origin / Destination':'Origin', 'Latitude':Man_Utd_Latitude,'Longitude':Man_Utd_Longitude},index=[j-0.5])
            tables[i] = tables[i].append(utd,ignore_index=False)
        except:
            break

    tables[i] = tables[i].sort_index().reset_index(drop=True)

    path = []
    for k in range(int(len(tables[i])/2)):
        path.extend(["Path_"+str(k+1),"Path_"+str(k+1)])
    tables[i]['Path ID'] = path

Match: 0 Searching: Aston Villa Football Club Stadium
Location: Birmingham, England, United Kingdom (52.509220123291016, -1.885064959526062) 

Inserting coordinates into table
Match: 0 Man Utd are the home team

Match: 1 Searching: Brighton & Hove Albion Football Club Stadium
Location: Brighton & Hove, England, United Kingdom (50.861534118652344, -0.08248499780893326) 

Match: 2 Man Utd are the home team

Match: 3 Searching: Newcastle United Football Club Stadium
Location: Newcastle upon Tyne, England, United Kingdom (54.97566223144531, -1.6226149797439575) 

Match: 4 Man Utd are the home team

Match: 5 Man Utd are the home team

Match: 6 Searching: Everton Football Club Stadium
Location: England, United Kingdom (53.438541412353516, -2.96642804145813) 

Match: 7 Man Utd are the home team

Match: 8 Searching: Southampton Football Club Stadium
Location: Southampton, England, United Kingdom (50.9058952331543, -1.3909209966659546) 

Match: 9 Searching: West Ham United Football Club Stadium

In [7]:
# Competition names corresponding to opponent tables are assigned to the excel sheets.
sheet_names = ['Pre-season','Premier League','EFL Cup','UEFA Champions League']

# Creates an excel file with the desired dataframes.
with pd.ExcelWriter('travel.xlsx') as writer:
    for i in range(len(tables)):
        tables[i].to_excel(writer, sheet_name=sheet_names[i])